In [1]:
import os, sys
import gc
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
sys.path.append('../')
import torch

In [3]:
from get_algos import get_all_algos , run_experiment ,call_paths, plot_mean

In [4]:
import gym
from sb3_contrib import RecurrentPPO
from gym.wrappers.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack,VecVideoRecorder
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env

In [5]:
env_name='Breakout-v0'

### 1. Test
<ol>
    <li>Create the environment</li>
    <li>For each algorithm:</li>
    <ol><li>Load the model</li>
        <li>Evaluate the model for 5 sample iterations </li>
        <li>Output the score  for each algo and each iteration</li>
        <li>Render the video and store it </li>
    </ol>
</ol> 

In [6]:
algo_list=['PPO','RecurrentPPO']

### 3. Testing

In [12]:
def testing_model(algo_list, env_name,n_steps,vid_length):
    total_rewards={}
    for algo_name in reversed(algo_list):
        algo_rewards=[]
        if algo_name =='QRDQN' or 'TRPO':
            device='cpu'
        else:
            device='cuda' #GPU was unable to load the entire experience relay so moved to virtual memory on disk
        policy='CnnLstmPolicy' if algo_name=='RecurrentPPO' else 'CnnPolicy'
        log_path, render_path, model_path=call_paths(algo_name,env_name,n_steps)
        env=VecVideoRecorder(VecFrameStack(make_atari_env(env_name,monitor_dir=render_path),n_stack=6),render_path,record_video_trigger=lambda step: step == 0,video_length=vid_length)
        model_name=env_name+"_"+algo_name+"_model"
        model=eval(algo_name).load(os.path.join(model_path, model_name),env,device=device)
        for i in range(5):
            state=env.reset()
            epi_rewards=0
            while True:
                action,_=model.predict(state)
                state,reward,done,_=env.step(action)
                epi_rewards+=reward[0]
                env.render()
                if done:
                    algo_rewards.append(epi_rewards)
                    break
        total_rewards[algo_name]=algo_rewards
        del model
        env.close()
    return total_rewards

In [13]:
total_rewards=testing_model(algo_list, env_name,n_steps=5000000,vid_length=1000)

In [17]:
rand_env_reward={}
for algo in reversed(algo_list):
    render_path_random=os.path.join(env_name,algo+'_'+'5000000','render','random')
    env=Monitor(gym.make(env_name),directory=render_path_random,force=True)
    total_rewards=[]
    for i in range(5):
        epi_rewards=0
        env.reset()
        while True:
            action=env.action_space.sample()
            _,rewards,done,_=env.step(action)
            epi_rewards+=rewards
            env.render()
            if done:
                total_rewards.append(epi_rewards)
                break
    env.close()
    rand_env_reward[algo]=total_rewards         